# [SQL Murder Mystery](https://mystery.knightlab.com/)
*Can you find out whodunit?*

There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​. Start by retrieving the corresponding crime scene report from the police department’s database.

## Database Schema

<img alt='Database schema' src='img/ERD-2025-02-01-034356.png' width=50%>



### Install duckdb and magic_duckdb if not already installed

In [1]:
# %pip install duckdb magic_duckdb --upgrade --quiet

### Load our SQLITE database using DuckDB

In [2]:
%xmode minimal
%reload_ext magic_duckdb
%dql ATTACH 'data/sql-murder-mystery.db' (TYPE SQLITE);
%dql -t df_markdown FROM duckdb_databases();


Exception reporting mode: Minimal


| database_name      |   database_oid | path                       | comment   | tags   | internal   | type   | readonly   |
|:-------------------|---------------:|:---------------------------|:----------|:-------|:-----------|:-------|:-----------|
| memory             |           1146 |                            |           | {}     | False      | duckdb | False      |
| sql-murder-mystery |           1828 | data/sql-murder-mystery.db |           | {}     | False      | sqlite | False      |
| system             |              0 |                            |           | {}     | True       | duckdb | False      |
| temp               |           1820 |                            |           | {}     | True       | duckdb | False      |

### Specify the database to use and see what tables we have to work with

In [3]:
%dql USE 'sql-murder-mystery';
%dql -t df_markdown SELECT table_name, column_count, sql FROM duckdb_tables();

| table_name             |   column_count | sql                                                                                                                                                                                               |
|:-----------------------|---------------:|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| crime_scene_report     |              4 | CREATE TABLE crime_scene_report(date BIGINT, "type" VARCHAR, description VARCHAR, city VARCHAR);                                                                                                  |
| drivers_license        |              9 | CREATE TABLE drivers_license(id BIGINT PRIMARY KEY, age BIGINT, height BIGINT, eye_color VARCHAR, hair_color VARCHAR, gender VARCHAR, plate_number VARCHAR, car_make VARCHAR, car_model VARCHAR); |
| facebook_event_checkin |              4 | CREATE TABLE facebook_event_checkin(person_id BIGINT, event_id BIGINT, event_name VARCHAR, date BIGINT);                                                                                          |
| interview              |              2 | CREATE TABLE interview(person_id BIGINT, transcript VARCHAR);                                                                                                                                     |
| get_fit_now_member     |              5 | CREATE TABLE get_fit_now_member(id VARCHAR PRIMARY KEY, person_id BIGINT, "name" VARCHAR, membership_start_date BIGINT, membership_status VARCHAR);                                               |
| get_fit_now_check_in   |              4 | CREATE TABLE get_fit_now_check_in(membership_id VARCHAR, check_in_date BIGINT, check_in_time BIGINT, check_out_time BIGINT);                                                                      |
| solution               |              2 | CREATE TABLE solution("user" BIGINT, "value" VARCHAR);                                                                                                                                            |
| income                 |              2 | CREATE TABLE income(ssn VARCHAR PRIMARY KEY, annual_income BIGINT);                                                                                                                               |
| person                 |              6 | CREATE TABLE person(id BIGINT PRIMARY KEY, "name" VARCHAR, license_id BIGINT, address_number BIGINT, address_street_name VARCHAR, ssn VARCHAR);                                                   |

### Alternatively

In [4]:
%dql -t df_markdown SELECT table_name, column_name, data_type FROM duckdb_columns() WHERE database_name = 'sql-murder-mystery';

| table_name             | column_name           | data_type   |
|:-----------------------|:----------------------|:------------|
| crime_scene_report     | date                  | BIGINT      |
| crime_scene_report     | type                  | VARCHAR     |
| crime_scene_report     | description           | VARCHAR     |
| crime_scene_report     | city                  | VARCHAR     |
| drivers_license        | id                    | BIGINT      |
| drivers_license        | age                   | BIGINT      |
| drivers_license        | height                | BIGINT      |
| drivers_license        | eye_color             | VARCHAR     |
| drivers_license        | hair_color            | VARCHAR     |
| drivers_license        | gender                | VARCHAR     |
| drivers_license        | plate_number          | VARCHAR     |
| drivers_license        | car_make              | VARCHAR     |
| drivers_license        | car_model             | VARCHAR     |
| facebook_event_checkin | person_id             | BIGINT      |
| facebook_event_checkin | event_id              | BIGINT      |
| facebook_event_checkin | event_name            | VARCHAR     |
| facebook_event_checkin | date                  | BIGINT      |
| interview              | person_id             | BIGINT      |
| interview              | transcript            | VARCHAR     |
| get_fit_now_member     | id                    | VARCHAR     |
| get_fit_now_member     | person_id             | BIGINT      |
| get_fit_now_member     | name                  | VARCHAR     |
| get_fit_now_member     | membership_start_date | BIGINT      |
| get_fit_now_member     | membership_status     | VARCHAR     |
| get_fit_now_check_in   | membership_id         | VARCHAR     |
| get_fit_now_check_in   | check_in_date         | BIGINT      |
| get_fit_now_check_in   | check_in_time         | BIGINT      |
| get_fit_now_check_in   | check_out_time        | BIGINT      |
| solution               | user                  | BIGINT      |
| solution               | value                 | VARCHAR     |
| income                 | ssn                   | VARCHAR     |
| income                 | annual_income         | BIGINT      |
| person                 | id                    | BIGINT      |
| person                 | name                  | VARCHAR     |
| person                 | license_id            | BIGINT      |
| person                 | address_number        | BIGINT      |
| person                 | address_street_name   | VARCHAR     |
| person                 | ssn                   | VARCHAR     |

### Collect some stats on the `crime_scene_report` table

In [5]:
%dql -t df_markdown SUMMARIZE crime_scene_report;

| column_name   | column_type   | min      | max                                         |   approx_unique |        avg |     std |      q25 |      q50 |      q75 |   count |   null_percentage |
|:--------------|:--------------|:---------|:--------------------------------------------|----------------:|-----------:|--------:|---------:|---------:|---------:|--------:|------------------:|
| date          | BIGINT        | 20170101 | 20180501                                    |             426 | 2.0173e+07 | 4160.93 | 20170433 | 20170905 | 20174384 |    1228 |                 0 |
| type          | VARCHAR       | arson    | theft                                       |              10 |            |         |          |          |          |    1228 |                 0 |
| description   | VARCHAR       |          | ‘tis love, that makes the world go round!”’ |             797 |            |         |          |          |          |    1228 |                 0 |
| city          | VARCHAR       | Aberdeen | Youngstown                                  |             423 |            |         |          |          |          |    1228 |                 0 |

### Locate the crime report of interest

In [6]:
%%dql -t df_markdown
-- Find the report from January 15th, 2018
-- Type is murder, city is SQL City
FROM crime_scene_report
WHERE 
    date = 20180115 -- date is stored as int
    and city = 'SQL City' -- city of interest
    and type = 'murder' -- we're looking for the murder 

|     date | type   | description                                                                                                                                                                               | city     |
|---------:|:-------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------|
| 20180115 | murder | Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave". | SQL City |

There are two witnesses:
1. Lives at the last house on `Northwestern Dr` 
2. Named `Annabel`, lives somewhere on `Franklin Ave`

We'll need to discover who they are from the `person` table and review their `transcript`.

In [7]:
%%dql -t df_markdown
-- find our witness data
WITH find_northwestern_witness AS (
    FROM person
    WHERE address_street_name ilike '%northwestern%'
    ORDER BY address_number DESC
    LIMIT 1
),
find_annabel_witness AS (
    FROM person
    WHERE name ilike '%annabel%' and address_street_name ilike '%franklin%'
),
witnesses AS (
    -- combine our witnesses together in a single result
    FROM find_northwestern_witness
    UNION
    FROM find_annabel_witness
)
-- and get their witness statement
FROM witnesses
LEFT JOIN interview 
    ON witnesses.id = interview.person_id

|    id | name           |   license_id |   address_number | address_street_name   |       ssn |   person_id | transcript                                                                                                                                                                                                                      |
|------:|:---------------|-------------:|-----------------:|:----------------------|----------:|------------:|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 16371 | Annabel Miller |       490173 |              103 | Franklin Ave          | 318771143 |       16371 | I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.                                                                                                           |
| 14887 | Morty Schapiro |       118009 |             4919 | Northwestern Dr       | 111564949 |       14887 | I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W". |

From the witness transcripts we have:
- Annabel recognizing the killer from her gym when she was working out on `January 9th (2018)`.
- Morty telling us the following about the suspect:
  - Gender: `male`
  - Had a gym bag from `Get Fit Now Gym`
  - Membership number on the bag started with `48Z`
  - Only `gold` members have those bags.
  - Got into a car with a license plate that had `H42W` in it.

Let's see where our two witnesses were using their Facebook check-ins `facebook_event_checkin`.

In [8]:
%%dql -t df_markdown
-- Annabel and Morty 
FROM facebook_event_checkin f
LEFT JOIN person p
    ON f.person_id = p.id
WHERE person_id in (16371, 14887)

|   person_id |   event_id | event_name             |     date |    id | name           |   license_id |   address_number | address_street_name   |       ssn |
|------------:|-----------:|:-----------------------|---------:|------:|:---------------|-------------:|-----------------:|:----------------------|----------:|
|       14887 |       4719 | The Funky Grooves Tour | 20180115 | 14887 | Morty Schapiro |       118009 |             4919 | Northwestern Dr       | 111564949 |
|       16371 |       4719 | The Funky Grooves Tour | 20180115 | 16371 | Annabel Miller |       490173 |              103 | Franklin Ave          | 318771143 |

Let's see if we can find out more about the owner of the car with the plate `H42W` by looking at the `drivers_license` table and joining that data with the `person` table

In [9]:
%%dql -t df_markdown
-- drivers of interest
FROM drivers_license dl
LEFT JOIN person p
    ON dl.id = p.license_id
WHERE plate_number ILIKE '%h42W%';

|     id |   age |   height | eye_color   | hair_color   | gender   | plate_number   | car_make   | car_model   |   id_1 | name           |   license_id |   address_number | address_street_name   |       ssn |
|-------:|------:|---------:|:------------|:-------------|:---------|:---------------|:-----------|:------------|-------:|:---------------|-------------:|-----------------:|:----------------------|----------:|
| 183779 |    21 |       65 | blue        | blonde       | female   | H42W0X         | Toyota     | Prius       |  78193 | Maxine Whitely |       183779 |              110 | Fisk Rd               | 137882671 |
| 423327 |    30 |       70 | brown       | brown        | male     | 0H42W2         | Chevrolet  | Spark LS    |  67318 | Jeremy Bowers  |       423327 |              530 | Washington Pl, Apt 3A | 871539279 |
| 664760 |    21 |       71 | black       | black        | male     | 4H42WR         | Nissan     | Altima      |  51739 | Tushar Chandra |       664760 |              312 | Phi St                | 137882671 |

Let's see if any of the gym members have any overlaps with the drivers of interest.

In [10]:
%%dql -t df_markdown
FROM get_fit_now_check_in gci
LEFT JOIN get_fit_now_member gm
    ON gci.membership_id = gm.id
LEFT JOIN person p
    ON gm.person_id = p.id
WHERE 
    (gm.membership_status = 'gold' AND gm.id LIKE '48Z%')
AND gci.check_in_date = 20180109
OR p.id in (14887, 16371)


| membership_id   |   check_in_date |   check_in_time |   check_out_time | id    |   person_id | name           |   membership_start_date | membership_status   |   id_1 | name_1         |   license_id |   address_number | address_street_name   |       ssn |
|:----------------|----------------:|----------------:|-----------------:|:------|------------:|:---------------|------------------------:|:--------------------|-------:|:---------------|-------------:|-----------------:|:----------------------|----------:|
| 90081           |        20180109 |            1600 |             1700 | 90081 |       16371 | Annabel Miller |                20160208 | gold                |  16371 | Annabel Miller |       490173 |              103 | Franklin Ave          | 318771143 |
| 48Z7A           |        20180109 |            1600 |             1730 | 48Z7A |       28819 | Joe Germuska   |                20160305 | gold                |  28819 | Joe Germuska   |       173289 |              111 | Fisk Rd               | 138909730 |
| 48Z55           |        20180109 |            1530 |             1700 | 48Z55 |       67318 | Jeremy Bowers  |                20160101 | gold                |  67318 | Jeremy Bowers  |       423327 |              530 | Washington Pl, Apt 3A | 871539279 |

That gives us two suspects:
1. person.id `28819`, Joe Germuska at 111 Fisk Rd.
2. person.id `67318`, Jeremy Bowers at 530 Washington Pl, Apt 3A

Let's check their Facebook check-in events. 

In [11]:
%%dql -t df_markdown
-- Joe and Jeremy
FROM facebook_event_checkin f
LEFT JOIN person p
    ON f.person_id = p.id
WHERE person_id in (28819, 67318)

|   person_id |   event_id | event_name             |     date |    id | name          |   license_id |   address_number | address_street_name   |       ssn |
|------------:|-----------:|:-----------------------|---------:|------:|:--------------|-------------:|-----------------:|:----------------------|----------:|
|       67318 |       1143 | SQL Symphony Concert   | 20171206 | 67318 | Jeremy Bowers |       423327 |              530 | Washington Pl, Apt 3A | 871539279 |
|       67318 |       4719 | The Funky Grooves Tour | 20180115 | 67318 | Jeremy Bowers |       423327 |              530 | Washington Pl, Apt 3A | 871539279 |

Joe had no Facebook check-ins where Jeremy was also at `The Funky Grooves Tour` event on the same day the murder and our witnesses. Assuming that are murder took place at that event than `Jeremy Bowers` is our likely murderer. 

In [12]:
%%dql -t df_markdown
INSERT INTO solution VALUES (1, 'Jeremy Bowers');
SELECT value FROM solution;

| value                                                                                                                                                                                                                                                                                                                                                                                              |
|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer. |

> Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.

In [13]:
%%dql -t df_markdown
-- get the interview transcript for Jermy Bowers
FROM interview
WHERE person_id = 67318

|   person_id | transcript                                                                                                                                                                                                                                       |
|------------:|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|       67318 | I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017. |

### Let's find the woman who hired `Jeremy Bowers`! 

In [14]:
%%dql -t df_markdown
WITH persons_attending_concert_3_times AS (
    -- find individuals who attended the 
    -- SQL Symphony Concert 3 times in December 2017
    FROM facebook_event_checkin f
    SELECT person_id
    WHERE 
        date BETWEEN 20171201 and 20171231
    AND event_name = 'SQL Symphony Concert'
    GROUP BY person_id
    HAVING COUNT(*) = 3
)
FROM drivers_license dl
INNER JOIN person p
    ON dl.id = p.license_id
INNER JOIN persons_attending_concert_3_times pac
    ON pac.person_id = p.id
INNER JOIN income i
    on p.ssn = i.ssn
WHERE 
    height between 65 and 67
AND gender = 'female'
AND car_make = 'Tesla'
AND hair_color = 'red'
AND car_model = 'Model S';


|     id |   age |   height | eye_color   | hair_color   | gender   |   plate_number | car_make   | car_model   |   id_1 | name             |   license_id |   address_number | address_street_name   |       ssn |   person_id |     ssn_1 |   annual_income |
|-------:|------:|---------:|:------------|:-------------|:---------|---------------:|:-----------|:------------|-------:|:-----------------|-------------:|-----------------:|:----------------------|----------:|------------:|----------:|----------------:|
| 202298 |    68 |       66 | green       | red          | female   |         500123 | Tesla      | Model S     |  99716 | Miranda Priestly |       202298 |             1883 | Golden Ave            | 987756388 |       99716 | 987756388 |          310000 |

`Miranda Priestly` fits the description provided by `Jeremy Bowers` let's check if we solved the mystery.

In [15]:
%%dql -t df_markdown
INSERT INTO solution VALUES (1, 'Miranda Priestly');
SELECT value FROM solution;

| value                                                                                                                                                        |
|:-------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne! |

> Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!

### Conclusion: `Miranda Priestly` hired the hitman `Jeremy Bowers`

---
# MISCELLANEOUS

I was curious how the solution checking logic was working, so I opened up the database file with [DBeaver](https://dbeaver.io/) and checked the Table Triggers which has `check_solution` that contains:

```sql
CREATE TRIGGER check_solution AFTER INSERT ON solution
    WHEN new.user==1
    BEGIN
        DELETE FROM solution;
        INSERT INTO solution VALUES (0,
        CASE WHEN hex(new.value)=='4A6572656D7920426F77657273' THEN "Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."
             WHEN hex(new.value)=='4D6972616E6461205072696573746C79' THEN "Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"
             ELSE "That's not the right person. Try again!"
        END
        );
    END;
```

Apparently DuckDB doesn't return `type = trigger` in the output.

In [16]:
%%dql -t df_markdown
FROM sqlite_master;

| type   | name                                  | tbl_name               |   rootpage | sql                                                                                                                                                                                               |
|:-------|:--------------------------------------|:-----------------------|-----------:|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| table  | crime_scene_report                    | crime_scene_report     |          0 | CREATE TABLE crime_scene_report(date BIGINT, "type" VARCHAR, description VARCHAR, city VARCHAR);                                                                                                  |
| table  | drivers_license                       | drivers_license        |          0 | CREATE TABLE drivers_license(id BIGINT PRIMARY KEY, age BIGINT, height BIGINT, eye_color VARCHAR, hair_color VARCHAR, gender VARCHAR, plate_number VARCHAR, car_make VARCHAR, car_model VARCHAR); |
| table  | facebook_event_checkin                | facebook_event_checkin |          0 | CREATE TABLE facebook_event_checkin(person_id BIGINT, event_id BIGINT, event_name VARCHAR, date BIGINT);                                                                                          |
| table  | interview                             | interview              |          0 | CREATE TABLE interview(person_id BIGINT, transcript VARCHAR);                                                                                                                                     |
| table  | get_fit_now_member                    | get_fit_now_member     |          0 | CREATE TABLE get_fit_now_member(id VARCHAR PRIMARY KEY, person_id BIGINT, "name" VARCHAR, membership_start_date BIGINT, membership_status VARCHAR);                                               |
| table  | get_fit_now_check_in                  | get_fit_now_check_in   |          0 | CREATE TABLE get_fit_now_check_in(membership_id VARCHAR, check_in_date BIGINT, check_in_time BIGINT, check_out_time BIGINT);                                                                      |
| table  | solution                              | solution               |          0 | CREATE TABLE solution("user" BIGINT, "value" VARCHAR);                                                                                                                                            |
| table  | income                                | income                 |          0 | CREATE TABLE income(ssn VARCHAR PRIMARY KEY, annual_income BIGINT);                                                                                                                               |
| table  | person                                | person                 |          0 | CREATE TABLE person(id BIGINT PRIMARY KEY, "name" VARCHAR, license_id BIGINT, address_number BIGINT, address_street_name VARCHAR, ssn VARCHAR);                                                   |
| index  | sqlite_autoindex_get_fit_now_member_1 | get_fit_now_member     |          0 | CREATE INDEX sqlite_autoindex_get_fit_now_member_1 ON get_fit_now_member USING  ();                                                                                                               |
| index  | sqlite_autoindex_income_1             | income                 |          0 | CREATE INDEX sqlite_autoindex_income_1 ON income USING  ();                                                                                                                                       |

In [17]:
%%dql -t df_markdown
-- reverse engineering the trigger
FROM person
SELECT *, hex(name) AS hex_value
WHERE 
    hex_value IN ('4A6572656D7920426F77657273', '4D6972616E6461205072696573746C79')

|    id | name             |   license_id |   address_number | address_street_name   |       ssn | hex_value                        |
|------:|:-----------------|-------------:|-----------------:|:----------------------|----------:|:---------------------------------|
| 67318 | Jeremy Bowers    |       423327 |              530 | Washington Pl, Apt 3A | 871539279 | 4A6572656D7920426F77657273       |
| 99716 | Miranda Priestly |       202298 |             1883 | Golden Ave            | 987756388 | 4D6972616E6461205072696573746C79 |

In [18]:
%dql --close

Related Github for [SQL Murder Mysteries](https://github.com/NUKnightLab/sql-mysteries)